In [3]:
from transformers import AutoTokenizer,GPT2Tokenizer

model_name = "gpt2"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")

In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
import pandas as pd
df=pd.read_csv('./data/books.csv').drop(columns=['Unnamed: 0', 'input_ids', 'att_mask'])

In [7]:
from transformers import AutoModelForSequenceClassification, AutoConfig

model_name='gpt2'
#config=AutoConfig.from_pretrained(model_name)
pt_model = AutoModelForSequenceClassification.from_pretrained("openai-community/gpt2", num_labels=df.genre.nunique())

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
pt_model.eval()
pt_model.score.train()

Linear(in_features=768, out_features=10, bias=False)

In [9]:
from datasets import Dataset, DatasetDict
import numpy as np

df.summary=df.summary.map(lambda x: x.strip().lower())
genre_dict={}
for i, g in enumerate(df.genre.unique()):
    genre_dict[g]=i
df['label']=df.genre.map(lambda x: genre_dict[x])


df=df.iloc[np.random.permutation(df.shape[0])].reset_index(drop=True)
split=int(df.shape[0]*0.9)

train_df=df.iloc[:split]
val_df=df.iloc[split:].reset_index(drop=True)


traindataset= Dataset.from_pandas(train_df, split='train')
valdataset=Dataset.from_pandas(val_df, split='val')
dataset=DatasetDict({
    "train": traindataset,
    "valid": valdataset})

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def tokenize_dataset(dataset):
     return tokenizer(dataset["summary"])
def trim_dataset(dataset):
     dataset['attention_mask']=[x[:1024] for x in dataset['attention_mask']]
     dataset['input_ids']=[x[:1024] for x in dataset['input_ids']]
     return dataset
dataset = dataset.map(tokenize_dataset, batched=True)
dataset = dataset.map(trim_dataset, batched=True)

Map: 100%|██████████| 466/466 [00:00<00:00, 3655.60 examples/s]


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
     output_dir="./gpt2/results/",
     learning_rate=2e-5,
     per_device_train_batch_size=8,
     per_device_eval_batch_size=8,
     num_train_epochs=2,
 )

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=pt_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kpuchalskixiv. Use `wandb login --relogin` to force relogin


  0%|          | 0/1048 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 415.94 MiB is free. Including non-PyTorch memory, this process has 4.78 GiB memory in use. Of the allocated memory 4.31 GiB is allocated by PyTorch, and 344.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)